In [1]:
import xarray as xr
import numpy as np
import xrft
from xgcm.grid import Grid
from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
from glob import glob
from pyresample.bilinear import NumpyBilinearResampler
from pyresample import geometry
from matplotlib.colors import LogNorm
from oceanmodes import baroclinic
from fastjmd95 import jmd95numba

In [2]:
RC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/RC.data', dtype='>f4')
RF =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/RF.data', dtype='>f4')
DXG =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/DXG.data', dtype='>f4').reshape(2160,2160)
DYG =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/DYG.data', dtype='>f4').reshape(2160,2160)
DXC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/DXC.data', dtype='>f4').reshape(2160,2160)
DYC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/DYC.data', dtype='>f4').reshape(2160,2160)
XC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/XC.data', dtype='>f4').reshape(2160,2160)
YC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/YC.data', dtype='>f4').reshape(2160,2160)
XG =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/XG.data', dtype='>f4').reshape(2160,2160)
YG =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/YG.data', dtype='>f4').reshape(2160,2160)
RAC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/RAC.data', dtype='>f4').reshape(2160,2160)
hFacC =np.fromfile('/swot/SUM01/spencer/swot_agulhas_test/hFacC.data', dtype='>f4').reshape(-1,2160,2160)[0,:,:]

In [3]:
Temp = xr.open_zarr('/swot/SUM02/spencer/swot_agulhas_surf/temp.zarr')
Salt = xr.open_zarr('/swot/SUM02/spencer/swot_agulhas_surf/salt.zarr')

In [4]:
Density = jmd95numba.rho(Salt.Salt, Temp.Theta, 0)
Density = Density.assign_coords(depth=('k', RC))
N2 = -9.81 *Density.mean('time').isel(i=slice(900,1100),j=slice(900,1100)).mean(['i','j']).diff('k')/Density.depth.diff('k')/1025
fm = 2*2*np.pi/24/3600*np.sin(-43*np.pi/180)

In [5]:
phi, RdS, vdS = baroclinic.neutral_modes_from_N2_profile(-RF[11:61], N2[10:60], fm,num_eigen=11,init_vector=None,num_Lanczos=None,iteration=None,tolerance=0)

In [6]:
vels_filtered = xr.open_zarr('/swot/SUM03/spencer/hourly_filt/filtered_vels.zarr')
vels_unfiltered = xr.open_zarr('/swot/SUM03/spencer/hourly_filt/unfiltered_vels.zarr')
eta_filtered = xr.open_zarr('/swot/SUM03/spencer/hourly_filt/filtered_eta_nom.zarr')
eta_unfiltered = xr.open_zarr('/swot/SUM03/spencer/hourly_filt/unfiltered_eta_nom.zarr')

uvel_eul = xr.open_zarr('/swot/SUM01/spencer/swot_agulhas_surf/uvel_centered.zarr').sel(k=0)
vvel_eul = xr.open_zarr('/swot/SUM01/spencer/swot_agulhas_surf/vvel_centered.zarr').sel(k=0)
steric2 = xr.open_zarr('/swot/SUM02/spencer/steric_inc_mean.zarr')
steric_mean = xr.open_zarr('/swot/SUM02/spencer/swot_agulhas_surf/steric_mean.zarr')

In [7]:
#uvel_eul.isel(time=slice(1,250)).time
vels_filtered['time']=uvel_eul.isel(time=slice(42,42+170)).time
vels_unfiltered['time']=uvel_eul.isel(time=slice(42,42+170)).time
eta_filtered['time']=uvel_eul.isel(time=slice(42,42+170)).time

In [8]:
ssh_unfilt = (steric2-steric_mean).isel(time=slice(0,254))
ssh_unfilt['time']=uvel_eul.isel(time=slice(1,170+42*2+1)).time

In [9]:
rossby_radius = xr.DataArray(RdS,dims=['Mode'])
rossby_radius

<xarray.DataArray (Mode: 11)>
array([3.18826406e+09, 1.11589611e+04, 6.10257796e+03, 3.94067152e+03,
       2.76847388e+03, 2.13333784e+03, 1.75954966e+03, 1.54576930e+03,
       1.36815190e+03, 1.20269432e+03, 1.07375616e+03])
Dimensions without coordinates: Mode

In [10]:
merged_ds = uvel_eul.isel(time=slice(1,254)).rename({'U':'unfiltered_u'}).merge(
    vels_filtered.astype('float32').rename({'u':'lagfiltered_u','v':'lagfiltered_v'}))
merged_ds = merged_ds.merge(vvel_eul.isel(time=slice(1,254)).rename({'V':'unfiltered_v'}))
merged_ds = merged_ds.merge(eta_filtered.astype('float32').rename({'eta':'lagfiltered_ssh'}))
merged_ds = merged_ds.merge(ssh_unfilt.drop_vars('niter').rename({'steric': 'unfiltered_steric_ssh'}))
merged_ds = merged_ds.merge(rossby_radius.rename('Rossby_radius'))
merged_ds = merged_ds.assign_coords({'Mode':['bartotropic mode','1st baroclinic mode','2nd baroclinic mode',
                                 '3rd baroclinic mode','4th baroclinic mode', '5th baroclinic mode',
                                '6th baroclinic mode','7th baroclinic mode','8th baroclinic mode',
                                '9th baroclinic mode','10th baroclinic mode']})

/home/spencerj/.conda/envs/pangeo2/lib/python3.7/site-packages/xarray/core/indexing.py:1381: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/spencerj/.conda/envs/pangeo2/lib/python3.7/site-packages/dask/array/core.py:4048: PerformanceWarning: Increasing number of chunks by factor of 12
  **blockwise_kwargs,
/home/spencerj/.conda/envs/pangeo2/lib/python3.7/site-packages/dask/array/core.py:4048: PerformanceWarning: Increasing number of chunks by factor of 12
  **blockwise_kwargs,
/home/spencerj/.conda/envs/pangeo2/lib/python3.7/site-packages/xarray/core/indexing.py:1381: PerformanceWarning: Slicing is producing a large chunk. To 

In [11]:
merged_ds.x0.attrs['units']='degrees_east'
merged_ds.x0.attrs['long_name']='longitude'
merged_ds.y0.attrs['units']='degrees_north'
merged_ds.y0.attrs['long_name']='latitude'
merged_ds.z0.attrs['units']='meters'
merged_ds.z0.attrs['long_name']='depth'
merged_ds=merged_ds.drop_vars(['x','y','face','niter'])

merged_ds = merged_ds.assign_coords(DXG=(["j_g","i"], DXG))
merged_ds = merged_ds.assign_coords(DYG=(["j","i_g"], DYG))
merged_ds = merged_ds.assign_coords(DXC=(["j","i_g"], DXC))
merged_ds = merged_ds.assign_coords(DYC=(["j_g","i"], DYC))
merged_ds = merged_ds.assign_coords(XC=(["j","i"], XC))
merged_ds = merged_ds.assign_coords(YC=(["j","i"], YC))
merged_ds = merged_ds.assign_coords(XG=(["j_g","i_g"], XG))
merged_ds = merged_ds.assign_coords(YG=(["j_g","i_g"], YG))
merged_ds = merged_ds.assign_coords(RAC=(["j","i"], RAC))

merged_ds.Rossby_radius.attrs['long_name']='Rossby radius of deformation'
merged_ds.Rossby_radius.attrs['units']='meters'
merged_ds.Mode.attrs['long_name']='Mode for the associated rossby radius of deformation, obtained from solving eigenvalue problem for regional mean density profile'
merged_ds.time.attrs['units_comment']= 'ns since 2011-09-10'

merged_ds.unfiltered_u.attrs['units'] = 'm/s'
merged_ds.unfiltered_v.attrs['units'] = 'm/s'
merged_ds.lagfiltered_u.attrs['units'] = 'm/s'
merged_ds.lagfiltered_v.attrs['units'] = 'm/s'
merged_ds.lagfiltered_ssh.attrs['units'] = 'meters'
merged_ds.unfiltered_steric_ssh.attrs['units'] = 'meters'
merged_ds.DXC.attrs['units'] = 'meters'
merged_ds.DXG.attrs['units'] = 'meters'
merged_ds.DYC.attrs['units'] = 'meters'
merged_ds.DYG.attrs['units'] = 'meters'
merged_ds.XC.attrs['units'] = 'degrees_east'
merged_ds.XG.attrs['units'] = 'degrees_east'
merged_ds.YC.attrs['units'] = 'degrees_north'
merged_ds.YG.attrs['units'] = 'degrees_north'
merged_ds.RAC.attrs['units'] = 'm^2'


merged_ds.unfiltered_u.attrs['long_name'] = 'Unfiltered zonal velocity on tracer points'
merged_ds.unfiltered_v.attrs['long_name'] = 'Unfiltered meridional velocity on tracer points'
merged_ds.lagfiltered_u.attrs['long_name'] = 'Lagrangian filtered zonal velocity on tracer points'
merged_ds.lagfiltered_v.attrs['long_name'] = 'Lagrangian filtered meridional velocity on tracer points'
merged_ds.lagfiltered_ssh.attrs['long_name'] = 'Lagrangian filtered steric sea surface height anomaly'
merged_ds.unfiltered_steric_ssh.attrs['long_name'] = 'Steric sea surface height anomaly'
merged_ds.DXC.attrs['long_name'] = 'zonal distance between tracer points'
merged_ds.DXG.attrs['long_name'] = 'zonal distance between vorticity points'
merged_ds.DYC.attrs['long_name'] = 'meridional distance between tracer points'
merged_ds.DYG.attrs['long_name'] = 'meridional distance between vorticity points'
merged_ds.XC.attrs['long_name'] = 'Longitude of tracer points'
merged_ds.XG.attrs['long_name'] = 'Longitude of vorticity points'
merged_ds.YC.attrs['long_name'] = 'Latitude of tracer points'
merged_ds.YG.attrs['long_name'] = 'Latitude of vorticity points'
merged_ds.RAC.attrs['long_name'] = 'Area of tracer-centered cells'
merged_ds.lagfiltered_u.attrs['comment'] = 'Values are NaN for 42hours at the beginning and end of the timeseries'
merged_ds.lagfiltered_v.attrs['comment'] = 'Values are NaN for 42hours at the beginning and end of the timeseries'
merged_ds.lagfiltered_ssh.attrs['comment'] = 'Values are NaN for 42hours at the beginning and end of the timeseries'


from datetime import date
today = date.today()

merged_ds.attrs['title'] = '2 weeks of hourly surface velocity and ssh, both filtered and unfiltered'
merged_ds.attrs['institution'] = 'LDEO'
merged_ds.attrs['source'] = 'MITgcm LLC4320, with filtering methods applied to some of the data'
merged_ds.attrs['history'] = 'This version created ' + str(today) + ' by C. Spencer Jones'
merged_ds.attrs['references'] = 'Jones, C. S., Xiao, Q. , Abernathey, R. P. , Smith, K. S. Separating balanced and unbalanced flow at the surface of the Agulhas region using Lagrangian filtering (in prep)'
merged_ds.attrs['notes'] = 'The Lagrangian filtered fields are only defined for the middle 168 hours of the timeseries'
merged_ds= merged_ds.isel(time=slice(1,200))

In [16]:
xmina=900
xmaxa=-900

ymina=900
ymaxa=-900

xminb=900+500
xmaxb=-900+500

yminb=900+250
ymaxb=-900+250

dsA = merged_ds.isel(i=slice(xmina,xmaxa),j=slice(ymina,ymaxa),i_g=slice(xmina,xmaxa),j_g=slice(ymina,ymaxa))
dsB = merged_ds.isel(i=slice(xminb,xmaxb),j=slice(yminb,ymaxb),i_g=slice(xminb,xmaxb),j_g=slice(yminb,ymaxb))

In [18]:
dsA.chunk({"time": 1, "i": 360,"j":360}).to_zarr('/swot/SUM02/spencer/daily_filt/two_weeksA.zarr',consolidated=True)
dsB.chunk({"time": 1, "i": 360,"j":360}).to_zarr('/swot/SUM02/spencer/daily_filt/two_weeksB.zarr',consolidated=True)

In [13]:
merged_ds.chunk({"time": 1, "i": 180,"j":180}).to_zarr('/swot/SUM02/spencer/daily_filt/two_weeks.zarr',consolidated=True)